In [1]:
import cv2, numpy, time
from imutils.video import WebcamVideoStream, FPS
%matplotlib inline
import matplotlib as plt
import numpy as np

def nothing(x):
    pass
def take_first(l):
    return l[0]

In [49]:
#[color_index, "color_name", [under_hsv], [upper_hsv]]
hsvcolors = [
    [0, "Red", np.array((150, 0, 0)), np.array((200, 255, 255))],
    [1, "Green", np.array((0, 0, 0)), np.array((255, 255, 255))]
]
def find_out_color(img):
    cv2.imshow("Brick BGR", img)
    print(img.shape)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.imshow("Brick HSV", hsv)
    px_cnt = img.shape[0]*img.shape[1]
    for color_index, color_name, under_hls, upper_hls in hsvcolors:
        ir = cv2.inRange(hsv, under_hls, upper_hls)
        cv2.imshow(color_name, ir)
        cnz = cv2.countNonZero(ir)
        print(color_name, round(cnz/px_cnt*100, 2), "%")

In [ ]:
cap = cv2.VideoCapture(0)
cv2.namedWindow('thresh')
cv2.createTrackbar('T','thresh',0,255,nothing)
newest_brick = 0
newest_x = 10000
brick_colors = {}
for i in range(10000000):
    r = False
    while not r:
        r, fr = cap.read()
    cv2.imshow("input", fr)
    fr_orig = fr.copy()
    
    under_conveyor = numpy.array([150, 140, 140])
    upper_conveyor = numpy.array([255, 255, 255])
    conv = cv2.inRange(fr, under_conveyor, upper_conveyor)
    cv2.imshow("conv", conv)
    
    
    
    imgray = cv2.cvtColor(fr,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(imgray,cv2.getTrackbarPos('T','thresh'),255,cv2.THRESH_BINARY)
    cv2.imshow("thresh", thresh)
    
    not_conv = cv2.bitwise_not(conv)
    
    
    
    cv2.imshow("not_conv", not_conv)
    
    im2, contours, hierarchy = cv2.findContours(not_conv,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    #print(hierarchy)
    real_contours = []
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if x == y == 0:#whole pic as contour
            continue
        elif w < 200 or h < 200:
            continue
        elif w > 600 and h > 400:
            continue
        elif sum(cv2.mean(fr[x:x+w, y:y+h])) > 700:
            continue
        #cv2.rectangle(fr, (x, y), (x+w, y+h), (255,0,0), 2)
        mx = x + (w/2)
        my = y + (h/2)
        real_contours.append([x, y, w, h, mx, my])
    real_contours.sort(key=take_first)
    
    for x, y, w, h, mx, my in real_contours:
        if x < (newest_x - 32):
            print("found new brick!", (w, h))
            newest_brick += 1
            brick_colors[newest_brick] = find_out_color(fr_orig[y:y+h, x:x+w])
            
            
        cv2.rectangle(fr, (x, y), (x+w, y+h), (0,0,0), 2)
        """for i1, n1 in enumerate(real_contours):
        x1, y1, w1, h1, mx1, my1 = n1
        for i2, n2 in enumerate(real_contours):
            x2, y2, w2, h2, mx2, my2 = n2"""
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(fr,str(newest_brick),(x,y+64), font, 3,(0,0,0),2,cv2.LINE_AA)
    if real_contours:
        #print(len(contours), "\t", len(real_contours))
        newest_x = real_contours[0][0]
    cv2.line(fr,(newest_x,0),(newest_x,100),(255,0,0),5)
        
    cv2.imshow("output", fr)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cap.release()

found new brick! (237, 205)
(205, 237, 3)
Red 24.76 %
Green 100.0 %


In [4]:
cap.release()
